<a href="https://colab.research.google.com/github/atasipanda/fair_bipartite_matching/blob/main/Amazon_Employee_Access_Data_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This is an online editor. To see the outputs, hit **Runtime > Run all** option

In [ ]:
import random
"""
Function to create a random ranking of platforms
"""
def create_ranking(train_df):
    platforms = list(train_df.RESOURCE.unique())
    platform_ranks = {}
    random_platforms = random.sample(platforms, len(platforms))
    for i, p in enumerate(random_platforms):
        platform_ranks[p] = i + 1
    return platform_ranks

In [ ]:
"""
Function to calculate Delta value
Delta is the most number of groups an item can belong to
"""
def calc_Delta(item_group_df):
  Delta = 0
  item_groups = {}
  for ig in item_group_df.to_dict(orient='records'):
    item_groups[ig["MGR_ID"]] = ig["ROLE_FAMILY"]
    if len(ig["ROLE_FAMILY"]) > Delta:
      Delta = len(ig["ROLE_FAMILY"])
  return item_groups, Delta


In [ ]:
"""
Key: Resource
Value: Another dictionary say X
For X
Key: RoleFamily
Value: count of MGR_IDs in this RoleFamily that made a request for this particular Resource

Function to assign upper bounds per platform per group
"""
from collections import defaultdict
def assign_platform_bounds(data, n, m, g, k, item_groups):
  platform_bounds = defaultdict(dict)
  b = (k*n)/(m*g)
  spots = 0
  for r in data:
    role_fam = item_groups[r['MGR_ID']]
    for role in role_fam:
      if role not in platform_bounds[r["RESOURCE"]]: #and spots < n - 1:
        platform_bounds[r["RESOURCE"]][role] = b
  return platform_bounds

In [ ]:
"""
Creating edges from the data
"""
def create_edges(data, platform_ranks):
  edge_list = []
  item_edge_rank_dict = {}
  for r in data:
    edge_list.append((r["MGR_ID"], r["RESOURCE"]))
    edge_rank = platform_ranks[r["RESOURCE"]]
    if r["MGR_ID"] in item_edge_rank_dict:
      item_edge_rank_dict[r["MGR_ID"]].append([(r["MGR_ID"], r["RESOURCE"]), edge_rank])
    else:
      item_edge_rank_dict[r["MGR_ID"]] = []
      item_edge_rank_dict[r["MGR_ID"]].append([(r["MGR_ID"], r["RESOURCE"]), edge_rank])
  return edge_list, item_edge_rank_dict

In [ ]:
from scipy import linalg, optimize
from scipy.optimize import linprog
import copy

In [ ]:
"""
Finding a greedy maximal matching
"""
def find_maximal_matching(item_groups, platform_bounds, edge_list):
    A = copy.deepcopy(item_groups)
    P = copy.deepcopy(platform_bounds)
    E = copy.deepcopy(edge_list)
    M = []
    matched_nodes = set([])
    invalid = 0
    coef=[]
    count = 0
    for e in E:
        # print E[i]
        if e == 0:
          count = count+1
        coef.append(0)
    for i, e in enumerate(E):
        #print(e)
        if e == 0: #or x[i] <= norm:
            continue
        groups = A.get(e[0])
        for g in groups:
            if e[0] in matched_nodes:
                invalid = 1
                break
            if P.get(e[1]).get(g) <= 0:
                invalid = 1
                break
            P[e[1]][g] = P[e[1]][g] - 1
        if invalid == 1:
            invalid = 0
            continue
        M.append(e)
        coef[i] = 1
        matched_nodes.add(e[0])
    #print(coef)
    # return M, coef
    return coef

In [ ]:
from scipy.optimize import linprog
"""
The lp that is solved in step 1 of algorithm 4
"""
def delta_lp(item_groups, platform_bounds, edge_list, item_edge_rank_dict):
    A = copy.deepcopy(item_groups)
    P = copy.deepcopy(platform_bounds)
    E = copy.deepcopy(edge_list)
    obj = []
    lhs = []
    rhs = []
    platform_groups = {}
    zero_list = []
    one_list = []
    bounds = []
    m = len(platform_bounds)
    for e in E:
        zero_list.append(0)
        #bounds.append((0, 1))  # lower and upper bounds for each x value
    for item in item_edge_rank_dict:
      coef = copy.deepcopy(zero_list)
      if len(item_edge_rank_dict[item]) > 1:
        edge_rank_list = copy.deepcopy(item_edge_rank_dict[item])
        edge_rank_list.sort(key = lambda x: x[1])
      else:
        edge_rank_list = item_edge_rank_dict[item]
      for er in edge_rank_list:
        edge = er[0]
        rank = er[1]
        pos = E.index(edge)
        coef[pos] = -1
        lb = rank/(2*m)
        lhs.append(coef) #individual fairness constraints
        rhs.append(-1*lb) 
        # print(sum(coef))
        # print(-1*lb)
    for i, e in enumerate(E):
        one_list = copy.deepcopy(zero_list)
        one_list[i] = 1 #enforcing upper bound one more time
        lhs.append(one_list)
        rhs.append(1)
        obj.append(-1)
        initial_coef = list(zero_list)
        initial_coef[i] = 1
        item = e[0]
        platform = e[1]
        groups = A.get(item)
        for group in groups:
            pg = (platform, group)
            if pg not in platform_groups:
                platform_groups[pg] = list(initial_coef)
                # rhs.append(P.get(platform).get(group))
            else:
                platform_groups[pg][i] = 1
    for pg in platform_groups:
        #if P.get(pg[0]).get(pg[1])==None:
        #print(pg)
        #print(P.get(pg[0]))
        lhs.append(platform_groups[pg])
        rhs.append(P.get(pg[0]).get(pg[1]))
    opt = linprog(c=obj, A_ub=lhs, b_ub=rhs, bounds=(0,1))
    #print(sum(opt.x))
    return list(opt.x)
    #return sum(opt.x)

In [ ]:
import math
""" 
Algorithm 4
"""
def delta_log_alg(item_groups, platform_bounds, edge_list, item_edge_rank_dict, f):
    #print(E)
    A = copy.deepcopy(item_groups)
    P = copy.deepcopy(platform_bounds)
    E = copy.deepcopy(edge_list)
    d = len(E)
    x = delta_lp(A, P, E, item_edge_rank_dict)
    norm = sum(x)
    UB = norm
    match_count = 0
    alpha = []
    cost = 0
    while norm > f:
        norm_i=0
        for i, val in enumerate(x):
            if val == 0:
                E[i] = 0
        M = find_maximal_matching(A, P, E, f, x)
        # print('matching')
        # print(M)
        #if sum(M) == 0:
          #break
        #if sum(M) >= max:
          #max = sum(M)
        x_modified = [x[i] for i, e in enumerate(x) if x[i] != 0 and M[i] != 0]
        alpha_i = min(x_modified)
        #print("min value")
        #print(alpha_i)
        alpha.append(alpha_i)
        cost = cost + alpha_i*sum(M)
        for i, val in enumerate(x):
            if val != 0:
                x[i] = val - alpha_i * M[i]
        norm_i = sum(x)
        #if norm_i == norm:
          #print("residue before f")
          #print(norm_i)
          #break
        #else:
        norm = norm_i
        # if norm <= f:
        #   print("residue")
        #   print(norm_i)
        match_count = match_count + 1
    denominator = sum(alpha)
    if denominator!=0:
      output = cost/denominator
    else:
      print('Division by 0')
      output = cost
    return output, UB, match_count

In [ ]:
import pandas as pd
from io import StringIO
import re
from itertools import chain as flatten
import networkx as nx
from functools import partial
import holoviews as hv
import cvxpy as cp
import numpy as np
import math
from time import process_time
def main_function():
    df = pd.read_csv('emp_access_challenge_free.csv')  # load file as a data frame
    i = 1
    eps = 0.001
    output_file = 'result/output_1000.txt'  # replace with the file name you want
    while i <= 10:
        init_train_df = df.sample(n=1000)  # increase `n` as per sample size
        train_df = init_train_df.\
            drop_duplicates(subset=["MGR_ID", "RESOURCE","ROLE_FAMILY"], keep=False)   # dropping any duplicate rows
        final_edges = len(train_df)  # number of edges after duplicate rows are cleaned out
        file_name = 'result/emp_access_1000_'+ str(i) + '.csv'  # replace with the file name you want
        train_df.to_csv(file_name)  # saving the sample to a file
        g = train_df.ROLE_FAMILY.nunique()  # total number of unique groups
        n = train_df.MGR_ID.nunique()   # total number of unique items
        m = train_df.RESOURCE.nunique()  # total number of unique platforms
        k = math.ceil(m*g/n)   # factor to calculate the upper bound per group per platform
        platform_ranks = create_ranking(train_df)
        platforms_file_name = 'result/platform_ranks_1000_' + str(i) + '.txt'
        with open(platforms_file_name, 'w') as fp:  # storing the platform ranking
            for pr in platform_ranks:
                fp.write(str(pr) + ":" + str(platform_ranks[pr])+"\n")
        item_group_df = train_df.groupby('MGR_ID')['ROLE_FAMILY'].apply(lambda x: list(np.unique(x))).apply(list).reset_index()
        item_groups, Delta = calc_Delta(item_group_df)
        data = train_df.to_dict(orient='records')
        platform_bounds = assign_platform_bounds(data, n, m, g, k, item_groups)
        edge_list, item_edge_rank_dict = create_edges(data, platform_ranks)
        start_time = process_time()
        output, UB, match_count = delta_log_alg(item_groups, platform_bounds, edge_list, item_edge_rank_dict, eps)
        runtime = process_time() - start_time   # calculating the runtime
        log_val = math.log2(n/eps)
        approx = 2*(Delta+1)*(log_val+1)
        act = UB/output
        with open(output_file, 'a') as fp:  # storing the parameters and output
            fp.write("iteration: " + str(i) + "\n")
            fp.write("items: " + str(n) + ", platforms: " + str(m) + ", groups: "+ str(g) + ", edges: " + str(final_edges) +", UB: " + str(UB) + ", SOL: " + str(output) + "\n")
            fp.write("Delta: " + str(Delta) + ", UB/SOL: " + str(act) + ", approx:" + str(approx) +  ", # of matchings:" + str(match_count) + ", run-time: " + str(runtime) + "\n")
        i = i+1


In [ ]:
main_function()

In [ ]:
from google.colab import files
!zip -r /content/result_1000.zip /content/result
files.download('/content/result_1000.zip')